In [ ]:
import numpy as np                     #for working with array of numbers 
import pandas as pd                    #for manipulating datasets
import matplotlib.pyplot as plt       #creating figures, creates a plotting area in a figure, plots some lines in a plotting area, decorates the plot with labels.
import seaborn as sns                 #for high-level interface for drawing attractive and informative statistical graphics.
%matplotlib inline
import warnings                      #useful to alert the user of some condition in a program
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('online_retail.csv')  #load the dataset as Data
df.info()                              #return the dataset information
df.isnull().sum()  

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop(columns=['index'],inplace=True)

In [ ]:
len(df['Description'].unique())

## EXPLORALOTRY DATA ANALYSIS BELOW

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
Most_buyed_product=df.groupby(by='Description').agg({'Quantity':'sum'}).sort_values(by='Quantity',ascending=False).reset_index().head(15)
Most_buyed_product

In [ ]:
import plotly.express as px
fig=px.bar(Most_buyed_product,x='Description',y='Quantity',title='Top 15 Most Buyed Product')
fig

In [ ]:
# Average Sales per Country
Most_buyed_product = df.groupby("Country")["Quantity"].mean().sort_values()

plt.figure(figsize = (12, 7), dpi = 150)
sns.barplot(x = Most_buyed_product.values, y = Most_buyed_product.index)
plt.xlabel("Most_buyed_product")
plt.ylabel("Country")
plt.title("Description")

In [ ]:
top_products_countrywise=df.groupby(['Country','Description']).agg({'Quantity':'sum'}).reset_index()
top_products_countrywise

In [ ]:
countries=list(df['Country'].unique())
countries

In [ ]:
## Word clouds are used often in summarizing huge amounts of data, especially in research, documentation, manuals, academia, and other text data. They are also often used in marketing functions to have a quick overview of customer pain points from voice of the customer analysis.

from wordcloud import WordCloud

In [ ]:
text=[]
for i in countries:
    i=i.replace(' ','')
    text.append(i)
text=','.join(text)
wordcloud=WordCloud(width=1600,height=1200 , background_color='white').generate(text)
plt.title('Countries')
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

## Quarterly Sales below

In [ ]:
df["Quarterly"] = pd.PeriodIndex(df["InvoiceDate"], freq= 'Q')
quarterly_sales = df.groupby("Quarterly")["Quantity"].sum()

plt.figure(dpi = 100)
plt.pie(quarterly_sales, labels = ["2010 Q4", "2011 Q1", "2011 Q2", "2011 Q3", "2011 Q4"], autopct = "%1.1f%%")
plt.title("Quarterly Sales")

In [ ]:
fig, axes=plt.subplots(nrows=19,ncols=2 ,figsize=(25,150))
Top_Products_Countrywise=pd.DataFrame(columns=['country','Top_Product'])
for i,country in enumerate(countries):
    row=i//2
    col=i%2
    temp_df=top_products_countrywise[top_products_countrywise['Country']==country].sort_values(by='Quantity',ascending=False).head(10)
    Top_Products_Countrywise=pd.concat([Top_Products_Countrywise, pd.DataFrame({'country': [country], 'Top_Product': [temp_df.head(1)['Description'].values[0]]})], ignore_index=True)
    sns.barplot(temp_df,x='Quantity',y='Description',ax=axes[row,col])
    axes[row,col].set_title(country)

## Top Product For Each Country

In [ ]:
Top_Products_Countrywise

In [ ]:
df.groupby('InvoiceDate')['UnitPrice'].mean().plot(figsize=(12, 8))
plt.xlabel('Month')
plt.ylabel('Average Unit Price')
plt.title('Average Unit Price Trend')
plt.show()

## Top Product For Each Year

In [ ]:
from datetime import datetime

In [ ]:
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])
df['Invoice_year']=df['InvoiceDate'].dt.year
df['Invoice_month']=df['InvoiceDate'].dt.month
df['Invoice_hour']=df['InvoiceDate'].dt.hour

In [ ]:
df.sample(3)

In [ ]:
df['Invoice_year'].unique()

In [ ]:
year_2010_df=df[df['Invoice_year']==2010]
year_2010_df=year_2010_df.groupby(by='Description').agg({'Quantity':'sum'}).reset_index().sort_values(by='Quantity',ascending=False)
year_2010_df

In [ ]:
year_2011_df=df[df['Invoice_year']==2011]
year_2011_df=year_2011_df.groupby(by='Description').agg({'Quantity':'sum'}).reset_index().sort_values(by='Quantity',ascending=False)
year_2011_df

In [ ]:
fig,axes=plt.subplots(nrows=2,ncols=1,figsize=(20,20))
sns.barplot(year_2010_df.head(10),x='Quantity',y='Description',ax=axes[0])
axes[0].set_title('YEAR 2010')
sns.barplot(year_2011_df.head(10),x='Quantity',y='Description',ax=axes[1])
axes[1].set_title('YEAR 2011')

## Monthly Top Product

In [ ]:
top_products_monthly=df.groupby(by=['Invoice_month','Description']).agg({'Quantity':'sum'}).reset_index()
top_products_monthly

In [ ]:
top_products_monthly[top_products_monthly['Invoice_month']==1].sort_values(by='Quantity',ascending=False).head(10)

In [ ]:
fig,axes=plt.subplots(nrows=6,ncols=2,figsize=(25,80))
for i in range(0,12):
    row=i//2
    col=i%2
    temp_df=top_products_monthly[top_products_monthly['Invoice_month']==i+1].sort_values(by='Quantity',ascending=False).head(10)
    sns.barplot(temp_df,x='Quantity',y='Description',ax=axes[row,col])
    axes[row,col].set_title(i+1)

## Each Month Analysis

In [ ]:
df.groupby(by=['Invoice_month']).agg({'Quantity':'sum'}).reset_index()

In [ ]:
fig=px.line(df.groupby(by=['Invoice_month']).agg({'Quantity':'sum'}).reset_index(),x='Invoice_month',y='Quantity',title='Monthly sales')
fig.show()

## Findings: while doing the analysis, i discovered most of the sales was during November however, i wikk carry out more Analysis on the Hourly sales.

## HOURLY SALES ANALYSIS

In [ ]:
df.groupby(by='Invoice_hour').agg({'Quantity':'sum'}).reset_index()
fig=px.bar(df.groupby(by='Invoice_hour').agg({'Quantity':'sum'}).reset_index(),x='Invoice_hour',y='Quantity',title='Hourly Sale')
fig.show()

## The bar chart shows that the buysiest hour of the day is between 10:00 and 15:00

In [ ]:
Most_freq_cust=pd.DataFrame(df['CustomerID'].value_counts().head(10)).reset_index()
Most_freq_cust

In [ ]:
sns.barplot(Most_freq_cust,x='CustomerID',y='count')
plt.tick_params(axis='x',rotation=90)
plt.title('Most Frequent Customers')
plt.ylabel('No. of Orders')
plt.show()

In [ ]:
df['Total_amount_paid']=df['Quantity']*df['UnitPrice']
top_customers=df[df['CustomerID'].isin(Most_freq_cust['CustomerID'])][['CustomerID','Country','Description','Quantity','Total_amount_paid']]
top_customers

In [ ]:
top_customers[['CustomerID','Country']].drop_duplicates() 

In [ ]:
customer_Total_payments=df.groupby(by='CustomerID').agg({'Total_amount_paid':'sum'}).reset_index().sort_values(by='Total_amount_paid',ascending=False)
customer_Total_payments

## total amount paid by each customer

In [ ]:
Top_customer_total_payment=pd.merge(df[df['CustomerID'].isin(customer_Total_payments.head(10)['CustomerID'])][['CustomerID','Country']].drop_duplicates()
,customer_Total_payments.head(10),how='inner').sort_values(by='Total_amount_paid',ascending=False)
Top_customer_total_payment

In [ ]:
sns.barplot(Top_customer_total_payment,x='CustomerID',y='Total_amount_paid',hue='Country',dodge=False)
plt.tick_params(axis='x',rotation=90)
plt.title('Top customer_total_payment')
plt.show()

## CUSTOMERS PER EACH COUNTRY

In [ ]:
pd.DataFrame(df['Country'].value_counts()).reset_index()

In [ ]:
fig=px.bar(pd.DataFrame(df['Country'].value_counts()).reset_index(),y='Country',x='count')
fig.show()

## From the analysis so far, ataset Most of the Customers are from United kingdom

## Total paid by each customers

In [ ]:
Total_amount_paid_per_country=df.groupby('Country').agg({'Total_amount_paid':'sum'}).reset_index()
Total_amount_paid_per_country

In [ ]:
fig=px.bar(Total_amount_paid_per_country,x='Country',y='Total_amount_paid',title='Total amount paid per country ')
fig.show()

In [ ]:
## GeoPandas provides a high-level interface to the matplotlib library for making maps.
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
country_sales = df.groupby('Country')['Total_amount_paid'].sum().reset_index()
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.merge(country_sales, how='left', left_on='name', right_on='Country')


fig, ax = plt.subplots(1, 1, figsize=(15, 10))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
world.plot(column='Total_amount_paid', ax=ax, legend=True, cax=cax,legend_kwds={'label': "Total Amount Paid by Country"})
ax.set_title('Distribution of Sales Across Different Countries')
plt.show()

## Examine the correlation between numerical variables

In [ ]:
numeric_columns = df.select_dtypes(include=[np.number])
numeric_columns.dropna(inplace=True)
correlation_matrix = numeric_columns.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()